In [8]:
# import dependencies
import pandas as pd
import numpy as np
import json
import csv
import os
import time
from config import api_key_meetup
from pandas.io.json import json_normalize
import requests
import logging
import matplotlib.pyplot as plt
import seaborn as sns
#pd.DataFrame.from_dict(json_normalize(list), orient='columns')

-----------
### Cities
-----------

In [9]:
#load the states csv to a a pandas dataframe
result = {}

filepath = os.path.join("data_input","states_us.csv")

with open(filepath,'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        for column, value in row.items():
            result.setdefault(column, []).append(value)
            
states_df = pd.DataFrame.from_dict(result)
print(states_df.head())

  Abbreviation        Name
0           AL     Alabama
1           AK      Alaska
2           AZ     Arizona
3           AR    Arkansas
4           CA  California


In [10]:
# EXTRACT MEETUP CITIES FOR ALL THE STATES IN STATES_DF

# API host - CITIES
cities_list = []

#counter object
counter = 0

#loop through each state
for state in states_df['Abbreviation']:

    #construct the query
    base_url_city = "https://api.meetup.com"
    method = "/2/cities"

    #configuration
    params = {
            "country" : "US",
            "key" : api_key_meetup,
            "state" : state,
            "page" : 200
            }

    # request the url
    response = requests.get(base_url_city+method, params=params)

    # check if the requests are within X-RateLimits
    if int(response.headers['X-RateLimit-Remaining']) <= 2:
        time.sleep(60)
      
    if counter % 5 == 0:
        print(f"{counter}:{response.url}")
    
    #increment the counter by 1
    counter += 1
    
    #store the json in cities variable
    cities = response.json()['results']
    
    #loop through each city
    for city in cities:
        city['state_params'] = state
        city['country_params'] = params["country"]
        cities_list.append(city)

0:https://api.meetup.com/2/cities?country=US&key=6178756c2f194627413177171e5f3f2f&state=AL&page=200
5:https://api.meetup.com/2/cities?country=US&key=6178756c2f194627413177171e5f3f2f&state=CO&page=200
10:https://api.meetup.com/2/cities?country=US&key=6178756c2f194627413177171e5f3f2f&state=HI&page=200
15:https://api.meetup.com/2/cities?country=US&key=6178756c2f194627413177171e5f3f2f&state=KS&page=200
20:https://api.meetup.com/2/cities?country=US&key=6178756c2f194627413177171e5f3f2f&state=MA&page=200
25:https://api.meetup.com/2/cities?country=US&key=6178756c2f194627413177171e5f3f2f&state=MT&page=200
30:https://api.meetup.com/2/cities?country=US&key=6178756c2f194627413177171e5f3f2f&state=NM&page=200
35:https://api.meetup.com/2/cities?country=US&key=6178756c2f194627413177171e5f3f2f&state=OK&page=200
40:https://api.meetup.com/2/cities?country=US&key=6178756c2f194627413177171e5f3f2f&state=SD&page=200
45:https://api.meetup.com/2/cities?country=US&key=6178756c2f194627413177171e5f3f2f&state=VA&p

In [11]:
response.headers

{'Content-Type': 'application/json;charset=utf-8', 'Server': 'Apache-Coyote/1.1', 'X-Meetup-server': 'b53e30fe22c4', 'X-Meetup-Request-ID': 'e911fd44-95ba-4f1a-a03f-d9e2e99c69b0', 'X-RateLimit-Limit': '30', 'X-RateLimit-Remaining': '8', 'X-RateLimit-Reset': '4', 'X-OAuth-Scopes': 'basic', 'X-Accepted-OAuth-Scopes': 'basic', 'ETag': '"504775c9fe4e950398e32302be745fa7-gzip"', 'Content-Encoding': 'gzip', 'Content-Length': '7540', 'Accept-Ranges': 'bytes', 'Date': 'Wed, 06 Dec 2017 19:51:50 GMT', 'Via': '1.1 varnish', 'Connection': 'keep-alive', 'X-Served-By': 'cache-dfw18627-DFW', 'X-Cache': 'MISS', 'X-Cache-Hits': '0', 'X-Timer': 'S1512589910.270615,VS0,VE74', 'Vary': 'Accept-Encoding,User-Agent,Accept-Language'}

    X-RateLimit-Limit	The maximum number of requests that can be made in a window of time
    X-RateLimit-Remaining	The remaining number of requests allowed in the current rate limit window
    X-RateLimit-Reset	The number of seconds until the current rate limit window resets

In [12]:
df_city = pd.DataFrame.from_dict(cities_list)

In [15]:
# exclude alaska and hawaii states
df_city = df_city[~df_city['state'].isin(['HI', 'AK'])]

In [16]:
df_city.to_csv("data_output/df_meetup_cities.csv", encoding="utf-8", index=False)

In [17]:
df_city.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9203 entries, 0 to 9494
Data columns (total 13 columns):
city                      9203 non-null object
country                   9203 non-null object
country_params            9203 non-null object
distance                  9203 non-null float64
id                        9203 non-null int64
lat                       9203 non-null float64
localized_country_name    9203 non-null object
lon                       9203 non-null float64
member_count              9203 non-null int64
ranking                   9203 non-null int64
state                     9203 non-null object
state_params              9203 non-null object
zip                       9203 non-null object
dtypes: float64(3), int64(3), object(7)
memory usage: 1006.6+ KB
